In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.metrics import classification_report
import warnings
import warnings
from sklearn.cluster import KMeans

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
df_tr=pd.read_csv('training.csv')

In [3]:
df_tr.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [4]:
df_tr.shape

(4920, 133)

In [5]:
df_tr.iloc[-1]

itching                        0
skin_rash                      1
nodal_skin_eruptions           0
continuous_sneezing            0
shivering                      0
                          ...   
inflammatory_nails             0
blister                        1
red_sore_around_nose           1
yellow_crust_ooze              1
prognosis               Impetigo
Name: 4919, Length: 133, dtype: object

In [6]:
df_tt=pd.read_csv('testing.csv')

In [7]:
df_tt.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Allergy
2,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,GERD
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Chronic cholestasis
4,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Drug Reaction


In [8]:
symp=[]
disease=[]
for i in range(len(df_tr)):
    symp.append(df_tr.columns[df_tr.iloc[i]==1].to_list())
    disease.append(df_tr.iloc[i,-1])

In [9]:
symp[0]

['itching', 'skin_rash', 'nodal_skin_eruptions', 'dischromic _patches']

In [10]:
disease[50]

'Peptic ulcer diseae'

# I- GET ALL SYMPTOMS

In [11]:
all_symp_col=list(df_tr.columns[:-1])
def clean_symp(sym):
    return sym.replace('_',' ').replace('.1','').replace('(typhos)','').replace('yellowish','yellow').replace('yellowing','yellow') 

In [12]:
all_symp=[clean_symp(sym) for sym in (all_symp_col)]

In [13]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Rudhreesh
[nltk_data]     Kumaar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
from nltk.corpus import wordnet as wn

In [15]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [16]:
## get all symptoms which do not have a synset
ohne_syns=[]
mit_syns=[]
for sym in all_symp:
    if not wn.synsets(sym) :
        ohne_syns.append(sym)
    else:
        mit_syns.append(sym)

In [17]:
len(mit_syns)

33

In [18]:
len(ohne_syns)

99

# II- Preprocess text

In [19]:
# %pip install spacy
# !python -m spacy download en_core_web_sm

In [20]:
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
nlp = spacy.load('en_core_web_sm')

In [21]:
def preprocess(doc):
    nlp_doc=nlp(doc)
    d=[]
    for token in nlp_doc:
        if(not token.text.lower()  in STOP_WORDS and  token.text.isalpha()):
            d.append(token.lemma_.lower() )
    return ' '.join(d)

In [22]:
def preprocess_sym(doc):
    nlp_doc=nlp(doc)
    d=[]
    for token in nlp_doc:
        if(not token.text.lower()  in STOP_WORDS and  token.text.isalpha()):
            d.append(token.lemma_.lower() )
    return ' '.join(d)

In [23]:
preprocess("skin peeling")

'skin peel'

In [24]:
all_symp_pr=[preprocess_sym(sym) for sym in all_symp]

In [25]:
#associate each preprocessed symp with the name of its original column
col_dict = dict(zip(all_symp_pr, all_symp_col))

# III- Syntactic Similarity



In [26]:
def jaccard_set(str1, str2):
    list1=str1.split(' ')
    list2=str2.split(' ')
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [27]:
#symp similarity with all the corpus
def syntactic_similarity(symp_t, corpus):
    most_sim = []
    poss_sym = []
    for symp in corpus:
        d = jaccard_set(symp_t, symp)
        most_sim.append(d)
    order = np.argsort(most_sim)[::-1].tolist()
    for i in order:
        if DoesExist(symp_t):
            return 1, [corpus[i]]
        if corpus[i] not in poss_sym and most_sim[i] != 0:
            poss_sym.append(corpus[i])
    if len(poss_sym):
        return 1, poss_sym
    else:
        return 0, None



In [28]:
import itertools
#Returns all the subsets of this set. This is a generator.
def powerset(seq):
    if len(seq) <= 1:
        yield seq
        yield []
    else:
        for item in powerset(seq[1:]):
            yield [seq[0]]+item
            yield item

In [29]:
#Sort list based on length
def sort(a):
    for i in range(len(a)):
        for j in range(i+1,len(a)):
            if len(a[j])>len(a[i]):
                a[i],a[j]=a[j],a[i]
    a.pop()
    return a

In [30]:
# find all permutations of a list
def permutations(s):
    permutations = list(itertools.permutations(s))
    return([' '.join(permutation) for permutation in permutations])

In [31]:
def DoesExist(txt):
    txt=txt.split(' ')
    combinations = [x for x in powerset(txt)]
    sort(combinations)
    for comb in combinations :
        #print(permutations(comb))
        for sym in permutations(comb):
            if sym in all_symp_pr:
                #print(sym)
                return sym
    return False
    

In [32]:
DoesExist('worried')

False

In [33]:
preprocess('really worried')

'worried'

In [34]:
syntactic_similarity(preprocess('nervous') ,all_symp_pr)

(0, None)

In [35]:
def check_pattern(inp,dis_list):
    import re
    pred_list=[]
    ptr=0
    patt = "^" + inp + "$"
    regexp = re.compile(inp)
    for item in dis_list:
        if regexp.search(item):
            pred_list.append(item)
    if(len(pred_list)>0):
        return 1,pred_list
    else:
        return ptr,None

In [36]:
check_pattern('nail',all_symp_pr)

(1, ['brittle nail', 'small dent nail', 'inflammatory nail'])

# IV- Semantic Similarity

In [37]:
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize
def WSD(word, context):
    sens=lesk(context, word)
    return sens

In [38]:
def semanticD(doc1,doc2):
    doc1_p=preprocess(doc1).split(' ')
    doc2_p=preprocess_sym(doc2).split(' ')
    score=0
    for tock1 in doc1_p:
        for tock2 in doc2_p:
            syn1 = WSD(tock1,doc1)
            syn2 = WSD(tock2,doc2)
            #syn1=wn.synset(t)
            if syn1 is not None and syn2 is not None :
                x=syn1.wup_similarity(syn2)
                if x is not None and x>0.1:
                    score+=x
    return score/(len(doc1_p)*len(doc2_p))

In [39]:
semanticD('anxiety','nervous')

0.2

In [40]:
syna=wn.synsets('anxiety')
syna[0].definition()

'(psychiatry) a relatively permanent state of worry and nervousness occurring in a variety of mental disorders, usually accompanied by compulsive behavior or attacks of panic'

In [41]:
synn=wn.synsets('nervous')
synn[0].definition()


'easily agitated'

In [42]:
synn[0].wup_similarity(syna[0])

0.2

In [43]:
anxiety_synsets = wn.synsets("brittle") 
nervous_synsets = wn.synsets("nervous") 
path=[]
wup=[]
lch=[]


for s1 in anxiety_synsets:
    for s2 in nervous_synsets:
        path.append(s1.path_similarity(s2))
        wup.append(s1.wup_similarity(s2))
        #lch.append(s1.lch_similarity(s2))
        

pd.DataFrame([path,wup],["path","wup"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
path,0.083333,0.083333,0.083333,0.083333,0.083333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333
wup,0.153846,0.153846,0.153846,0.153846,0.153846,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000


In [44]:
#Similarity with the corpus
def semantic_similarity(symp_t,corpus):
    max_sim=0
    most_sim=None
    for symp in corpus:
        d=semanticD(symp_t,symp)
        if d>max_sim:
            most_sim=symp
            max_sim=d
    return max_sim,most_sim

In [45]:
semantic_similarity('nervous',all_symp_pr)

(0.45, 'brittle nail')

In [46]:
all_symp_pr.sort()

In [47]:
all_symp_pr

['abdominal pain',
 'abnormal menstruation',
 'acidity',
 'acute liver failure',
 'alter sensorium',
 'anxiety',
 'belly pain',
 'blackhead',
 'bladder discomfort',
 'blister',
 'blood sputum',
 'bloody stool',
 'blur distorted vision',
 'breathlessness',
 'brittle nail',
 'bruise',
 'burn micturition',
 'chest pain',
 'chill',
 'cold hand feet',
 'coma',
 'congestion',
 'constipation',
 'continuous feel urine',
 'continuous sneezing',
 'cough',
 'cramp',
 'dark urine',
 'dehydration',
 'depression',
 'diarrhoea',
 'dischromic patch',
 'distention abdoman',
 'dizziness',
 'dry tingle lip',
 'enlarge thyroid',
 'excessive hunger',
 'extra marital contact',
 'family history',
 'fast heart rate',
 'fatigue',
 'fluid overload',
 'fluid overload',
 'foul smell urine',
 'headache',
 'high fever',
 'hip joint pain',
 'history alcohol consumption',
 'increase appetite',
 'indigestion',
 'inflammatory nail',
 'internal itching',
 'irregular sugar level',
 'irritability',
 'irritation anus',
 'i

In [48]:
from itertools import chain
from nltk.corpus import wordnet
def suggest_syn(sym):
    symp=[]
    synonyms = wordnet.synsets(sym)
    lemmas=[word.lemma_names() for word in synonyms]
    lemmas = list(set(chain(*lemmas)))
    for e in lemmas:
        res,sym1=semantic_similarity(e,all_symp_pr)
        if res!=0:
            symp.append(sym1)
    return list(set(symp))

In [49]:
suggest_syn('worried')

['blur distorted vision', 'malaise', 'brittle nail', 'unsteadiness']

In [50]:
#receives client_symptoms and returns a dataframe with 1 for associated symptoms
def OHV(cl_sym,all_sym):
    l=np.zeros([1,len(all_sym)])
    for sym in cl_sym:
        l[0,all_sym.index(sym)]=1
    return pd.DataFrame(l, columns =all_symp)
    

In [51]:
def contains(small, big):
    a=True
    for i in small:
        if i not in big:
            a=False
    return a

In [52]:
def possible_diseases(l):
    poss_dis=[]
    for dis in set(disease):
        if contains(l,symVONdisease(df_tr,dis)):
            poss_dis.append(dis)
    return poss_dis 

In [53]:
set(disease)

{'(vertigo) Paroymsal  Positional Vertigo',
 'AIDS',
 'Acne',
 'Alcoholic hepatitis',
 'Allergy',
 'Arthritis',
 'Bronchial Asthma',
 'Cervical spondylosis',
 'Chicken pox',
 'Chronic cholestasis',
 'Common Cold',
 'Dengue',
 'Diabetes ',
 'Dimorphic hemmorhoids(piles)',
 'Drug Reaction',
 'Fungal infection',
 'GERD',
 'Gastroenteritis',
 'Heart attack',
 'Hepatitis B',
 'Hepatitis C',
 'Hepatitis D',
 'Hepatitis E',
 'Hypertension ',
 'Hyperthyroidism',
 'Hypoglycemia',
 'Hypothyroidism',
 'Impetigo',
 'Jaundice',
 'Malaria',
 'Migraine',
 'Osteoarthristis',
 'Paralysis (brain hemorrhage)',
 'Peptic ulcer diseae',
 'Pneumonia',
 'Psoriasis',
 'Tuberculosis',
 'Typhoid',
 'Urinary tract infection',
 'Varicose veins',
 'hepatitis A'}

In [54]:
#receives an illness returns all symptoms
def symVONdisease(df,disease):
    ddf=df[df.prognosis==disease] 
    m2 = (ddf == 1).any()
    return m2.index[m2].tolist()
    

In [55]:
symVONdisease(df_tr,'Jaundice')

['itching',
 'vomiting',
 'fatigue',
 'weight_loss',
 'high_fever',
 'yellowish_skin',
 'dark_urine',
 'abdominal_pain']

# V- Prediction Model

In [56]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [57]:
X_train=df_tr.iloc[:,:-1]
X_test=df_tt.iloc[:,:-1]
y_train = df_tr.iloc[:,-1]
y_test = df_tt.iloc[:,-1]

In [58]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (4920, 132)
y_train shape: (4920,)
X_test shape: (41, 132)
y_test shape: (41,)


## KNN

In [59]:
# Define the range of hyperparameters to search
param_grid = {'n_neighbors': [1, 3, 5, 7, 9], 'weights': ['uniform', 'distance']}

# Create a KNN classifier object
knn = KNeighborsClassifier()

# Create a grid search object with cross-validation
knn_clf = GridSearchCV(knn, param_grid, cv=5)

# Fit the grid search object to the data
knn_clf.fit(X_train, y_train)

# Print the best hyperparameters and accuracy score
print("Best hyperparameters:", knn_clf.best_params_)
print("Best accuracy score:", knn_clf.best_score_)

# Generate predictions on the test set using the best estimator
y_pred = knn_clf.best_estimator_.predict(X_test)

# Print the classification report
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print("Classification report:\n", report)


Best hyperparameters: {'n_neighbors': 1, 'weights': 'uniform'}
Best accuracy score: 1.0
Classification report:
                                          precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00         1
                                   AIDS       1.00      1.00      1.00         1
                                   Acne       1.00      1.00      1.00         1
                    Alcoholic hepatitis       1.00      1.00      1.00         1
                                Allergy       1.00      1.00      1.00         1
                              Arthritis       1.00      1.00      1.00         1
                       Bronchial Asthma       1.00      1.00      1.00         1
                   Cervical spondylosis       1.00      1.00      1.00         1
                            Chicken pox       1.00      1.00      1.00         1
                    Chronic cholestasis       1.00      1.00      1.00       

## Decision Tree

In [60]:
# Define the Decision Tree Classifier
dt = DecisionTreeClassifier()

# Define the parameters to tune
param_grid = {
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the GridSearch object
dt_clf = GridSearchCV(dt, param_grid, cv=5)

# Fit the GridSearch object to the data
dt_clf.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters: ", dt_clf.best_params_)
print("Best score: ", dt_clf.best_score_)

# Get the best estimator from GridSearchCV
best_dt = dt_clf.best_estimator_

# Evaluate the best estimator on the test set
accuracy = best_dt.score(X_test, y_test)

# Print the accuracy score
print("Test set accuracy: ", accuracy)


Best parameters:  {'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 5}
Best score:  0.21463414634146344
Test set accuracy:  0.1951219512195122


In [61]:
# # Generate predictions on the test set using the best estimator
# y_pred = dt_clf.best_estimator_.predict(X_test)

# # Print the classification report
# from sklearn.metrics import classification_report
# report = classification_report(y_test, y_pred)
# print("Classification report:\n", report)

# # Get the feature importances
# importances = dt_clf.best_estimator_.feature_importances_

# # Print the feature importances
# print("Feature importances:")
# for i, feature in enumerate(X_train.columns):
#     print(f"{feature}: {importances[i]}")


## K-means

In [62]:
from sklearn.metrics import silhouette_score

kmeans = KMeans()

# create a parameter grid to search over
param_grid = {'n_clusters': [2, 3, 4, 5],
              'max_iter': [50, 100, 150, 200]}

# create a grid search object
km_clf = GridSearchCV(kmeans, param_grid=param_grid)

# fit the grid search object to your data
km_clf.fit(X_train,y_train)

# print the best parameters and score
print("Best parameters:", km_clf.best_params_)
print("Best score:", km_clf.best_score_)

# predict cluster labels for the test set using the best model
y_pred = km_clf.predict(X_test)

# calculate the silhouette score for the test set
silhouette = silhouette_score(X_test, y_pred)

print("Silhouette score:", silhouette)


Best parameters: {'max_iter': 100, 'n_clusters': 5}
Best score: -4604.536023493222
Silhouette score: 0.1210033599948168


In [63]:
# # Generate predictions on the test set using the best estimator
# y_pred = km_clf.best_estimator_.predict(X_test)

# # Print the classification report
# report = classification_report(y_test, y_pred)
# print("Classification report:\n", report)dd

## Mulitinomial Naive Bayes

The Multinomial Naive Bayes classifier is typically used for text classification tasks where the features are word counts or term frequencies. In contrast, the Gaussian Naive Bayes classifier assumes that the features are continuous variables that follow a Gaussian distribution.

In [64]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Create a Multinomial Naive Bayes classifier object
mnb = MultinomialNB()

# Define the hyperparameters to optimize
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0]
}

# Create a GridSearchCV object
nb_clf = GridSearchCV(mnb, param_grid, cv=5)

# Fit the GridSearchCV object on the training data
nb_clf.fit(X_train, y_train)

# Get the best hyperparameters and use them to train the classifier
best_params = nb_clf.best_params_
best_mnb = MultinomialNB(alpha=best_params['alpha'])
best_mnb.fit(X_train, y_train)

# Evaluate the classifier on the test data
y_pred = best_mnb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print the classification report
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 1.0
                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00         1
                                   AIDS       1.00      1.00      1.00         1
                                   Acne       1.00      1.00      1.00         1
                    Alcoholic hepatitis       1.00      1.00      1.00         1
                                Allergy       1.00      1.00      1.00         1
                              Arthritis       1.00      1.00      1.00         1
                       Bronchial Asthma       1.00      1.00      1.00         1
                   Cervical spondylosis       1.00      1.00      1.00         1
                            Chicken pox       1.00      1.00      1.00         1
                    Chronic cholestasis       1.00      1.00      1.00         1
                            Common Cold       1.00      1.00      1.00         1
             

## Random Forest

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# Define the parameter grid for the grid search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

# Create a random forest classifier object
rfc = RandomForestClassifier(random_state=42)

# Create a grid search object
rf_clf = GridSearchCV(estimator=rfc, param_grid=param_grid,
                          cv=5, n_jobs=-1, verbose=2)

# Fit the grid search object to the training data
rf_clf.fit(X_train, y_train)

# Get the best hyperparameters from the grid search
best_params = rf_clf.best_params_

# Create a new random forest classifier object with the best hyperparameters
rfc_best = RandomForestClassifier(random_state=42, **best_params)

# Fit the new random forest classifier to the training data
rfc_best.fit(X_train, y_train)

# Evaluate the performance of the new random forest classifier on the testing data
accuracy = rfc_best.score(X_test, y_test)
print("Accuracy:", accuracy)

# Print the classification report on the testing data
y_pred = rfc_best.predict(X_test)
print(classification_report(y_test, y_pred))

# # Print the feature importance
# feat_importances = rfc_best.feature_importances_
# print("Feature importances:", feat_importances)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Accuracy: 1.0
                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00         1
                                   AIDS       1.00      1.00      1.00         1
                                   Acne       1.00      1.00      1.00         1
                    Alcoholic hepatitis       1.00      1.00      1.00         1
                                Allergy       1.00      1.00      1.00         1
                              Arthritis       1.00      1.00      1.00         1
                       Bronchial Asthma       1.00      1.00      1.00         1
                   Cervical spondylosis       1.00      1.00      1.00         1
                            Chicken pox       1.00      1.00      1.00         1
                    Chronic cholestasis       1.00      1.00      1.00         1
                            Comm

## Top 5 probable diseases

In [66]:
diseases=df_tr.prognosis.unique()

In [67]:
classes=[x for x in range(0,41)] 
class_dict = {diseases: classes for diseases,
            classes in zip(diseases, classes)}
print(class_dict)

{'Fungal infection': 0, 'Allergy': 1, 'GERD': 2, 'Chronic cholestasis': 3, 'Drug Reaction': 4, 'Peptic ulcer diseae': 5, 'AIDS': 6, 'Diabetes ': 7, 'Gastroenteritis': 8, 'Bronchial Asthma': 9, 'Hypertension ': 10, 'Migraine': 11, 'Cervical spondylosis': 12, 'Paralysis (brain hemorrhage)': 13, 'Jaundice': 14, 'Malaria': 15, 'Chicken pox': 16, 'Dengue': 17, 'Typhoid': 18, 'hepatitis A': 19, 'Hepatitis B': 20, 'Hepatitis C': 21, 'Hepatitis D': 22, 'Hepatitis E': 23, 'Alcoholic hepatitis': 24, 'Tuberculosis': 25, 'Common Cold': 26, 'Pneumonia': 27, 'Dimorphic hemmorhoids(piles)': 28, 'Heart attack': 29, 'Varicose veins': 30, 'Hypothyroidism': 31, 'Hyperthyroidism': 32, 'Hypoglycemia': 33, 'Osteoarthristis': 34, 'Arthritis': 35, '(vertigo) Paroymsal  Positional Vertigo': 36, 'Acne': 37, 'Urinary tract infection': 38, 'Psoriasis': 39, 'Impetigo': 40}


In [68]:
y_train2=[]
for prognosis in df_tr.prognosis:
    y_train2.append(class_dict[prognosis])

In [69]:
y_test2=[]
for prognosis in df_tt.prognosis:
    y_test2.append(class_dict[prognosis])


In [70]:
import tensorflow as tf
from tensorflow import keras


model = keras.Sequential([
    keras.layers.Dense(132, input_shape=(132,), activation='relu'),
    keras.layers.Dense(41, activation='sigmoid'),
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(np.array(X_train), np.array(y_train2), epochs=1)

154/154 [==============================] - 1s 2ms/step - loss: 1.6683 - accuracy: 0.8795


In [71]:
model.evaluate(np.array(X_test),np.array(y_test2))

2/2 [==============================] - 0s 3ms/step - loss: 0.1642 - accuracy: 1.0000


[0.16422131657600403, 1.0]

In [72]:
pred=model.predict(X_test)
pred.shape

2/2 [==============================] - 0s 3ms/step


(41, 41)

In [73]:
#Extract Top five diseases
top_five_diseases=[]
for i in pred:
    top_five=[]
    top_five_indices=(-i).argsort()[:5]
    for j in top_five_indices:
        top_five.append(diseases[j])
    top_five_diseases.append(top_five)

In [74]:
#Extract Symptoms
all_symptoms=[]
for i in range(0,41):
    symptoms=[]
    for j in df_tt.columns:
        if((df_tt.iloc[i])[j]==1):
            symptoms.append(j)
    all_symptoms.append(symptoms)

In [75]:
data={'Symptoms':all_symptoms,'Predicted':top_five_diseases}
result=pd.DataFrame(data)

In [76]:
result

,Symptoms,Predicted
0,"[itching, skin_rash, nodal_skin_eruptions, dis...","[Fungal infection, Acne, Drug Reaction, Jaundi..."
1,"[continuous_sneezing, shivering, chills, water...","[Allergy, Pneumonia, AIDS, Fungal infection, M..."
2,"[stomach_pain, acidity, ulcers_on_tongue, vomi...","[GERD, Drug Reaction, Hypertension , Malaria, ..."
3,"[itching, vomiting, yellowish_skin, nausea, lo...","[Chronic cholestasis, Hepatitis D, hepatitis A..."
4,"[itching, skin_rash, stomach_pain, burning_mic...","[Drug Reaction, Fungal infection, GERD, Urinar..."
5,"[vomiting, indigestion, loss_of_appetite, abdo...","[Peptic ulcer diseae, Chronic cholestasis, Hea..."
6,"[muscle_wasting, patches_in_throat, high_fever...","[AIDS, Bronchial Asthma, Cervical spondylosis,..."
7,"[fatigue, weight_loss, restlessness, lethargy,...","[Diabetes , Hypoglycemia, Hyperthyroidism, Var..."
8,"[vomiting, sunken_eyes, dehydration, diarrhoea]","[Gastroenteritis, hepatitis A, Hepatitis D, Ch..."
9,"[fatigue, cough, high_fever, breathlessness, f...","[Bronchial Asthma, Pneumonia, Hepatitis C, Tub..."


In [77]:
for i in range(0,41):
    print("(",i+1,")"," Symptoms: ",end=" ")
    for j in all_symptoms[i]:
        print(j,end=", ")
    print("\n")
    print("       ","Predictions:",end=" ")
    for j in top_five_diseases[i]:
        print(j,end=", ")
    print("\n")

( 1 )  Symptoms:  itching, skin_rash, nodal_skin_eruptions, dischromic _patches, 

        Predictions: Fungal infection, Acne, Drug Reaction, Jaundice, Allergy, 

( 2 )  Symptoms:  continuous_sneezing, shivering, chills, watering_from_eyes, 

        Predictions: Allergy, Pneumonia, AIDS, Fungal infection, Malaria, 

( 3 )  Symptoms:  stomach_pain, acidity, ulcers_on_tongue, vomiting, cough, chest_pain, 

        Predictions: GERD, Drug Reaction, Hypertension , Malaria, (vertigo) Paroymsal  Positional Vertigo, 

( 4 )  Symptoms:  itching, vomiting, yellowish_skin, nausea, loss_of_appetite, abdominal_pain, yellowing_of_eyes, 

        Predictions: Chronic cholestasis, Hepatitis D, hepatitis A, Hepatitis C, Jaundice, 

( 5 )  Symptoms:  itching, skin_rash, stomach_pain, burning_micturition, spotting_ urination, 

        Predictions: Drug Reaction, Fungal infection, GERD, Urinary tract infection, Chronic cholestasis, 

( 6 )  Symptoms:  vomiting, indigestion, loss_of_appetite, abdominal

##  VI- SEVERITY / DESCRIPTION / PRECAUTION

In [78]:
severityDictionary=dict()
description_list = dict()
precautionDictionary=dict()

def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _description={row[0]:row[1]}
            description_list.update(_description)




def getSeverityDict():
    global severityDictionary
    with open('symptom_severity.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                _diction={row[0]:int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass


def getprecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _prec={row[0]:[row[1],row[2],row[3],row[4]]}
            precautionDictionary.update(_prec)


In [79]:
getSeverityDict()
getprecautionDict()
getDescription()

In [80]:
severityDictionary

{'itching': 1,
 'skin_rash': 3,
 'nodal_skin_eruptions': 4,
 'continuous_sneezing': 4,
 'shivering': 5,
 'chills': 3,
 'joint_pain': 3,
 'stomach_pain': 5,
 'acidity': 3,
 'ulcers_on_tongue': 4,
 'muscle_wasting': 3,
 'vomiting': 5,
 'burning_micturition': 6,
 'spotting_urination': 6,
 'fatigue': 4,
 'weight_gain': 3,
 'anxiety': 4,
 'cold_hands_and_feets': 5,
 'mood_swings': 3,
 'weight_loss': 3,
 'restlessness': 5,
 'lethargy': 2,
 'patches_in_throat': 6,
 'irregular_sugar_level': 5,
 'cough': 4,
 'high_fever': 7,
 'sunken_eyes': 3,
 'breathlessness': 4,
 'sweating': 3,
 'dehydration': 4,
 'indigestion': 5,
 'headache': 3,
 'yellowish_skin': 3,
 'dark_urine': 4,
 'nausea': 5,
 'loss_of_appetite': 4,
 'pain_behind_the_eyes': 4,
 'back_pain': 3,
 'constipation': 4,
 'abdominal_pain': 4,
 'diarrhoea': 6,
 'mild_fever': 5,
 'yellow_urine': 4,
 'yellowing_of_eyes': 4,
 'acute_liver_failure': 6,
 'fluid_overload': 4,
 'swelling_of_stomach': 7,
 'swelled_lymph_nodes': 6,
 'malaise': 6,
 'bl

In [81]:
def calc_condition(exp,days):
    sum=0
    for item in exp:
        if item in severityDictionary.keys():
            sum=sum+severityDictionary[item]
    if((sum*days)/(len(exp))>13):
        return 1
        print("You should take the consultation from doctor. ")
    else:
        return 0
        print("It might not be that bad but you should take precautions.")


# Chat

In [82]:
def getInfo():
    # name=input("Name:")
    print("Your Name \n\t\t\t\t\t\t",end="=>")
    name=input("")
    print("hello ",name)
    return str(name)

In [83]:
def related_sym(psym1):
    if len(psym1)==1:
        return psym1[0]
    print("searches related to input: ")
    for num,it in enumerate(psym1):
        print(num,")",clean_symp(it))
    if num!=0:
        print(f"Select the one you meant (0 - {num}):  ", end="")
        conf_inp = int(input(""))
    else:
        conf_inp=0

    disease_input=psym1[conf_inp]
    return disease_input

In [84]:
def main_sp(name,all_symp_col):
    #main Idea: At least two initial sympts to start with
    
    #get the 1st syp ->> process it ->> check_pattern ->>> get the appropriate one (if check_pattern==1 == similar syntaxic symp found)
    print("Enter the main symptom you are experiencing Mr/Ms "+name+"  \n\t\t\t\t\t\t",end="=>")
    sym1 = input("")
    sym1=preprocess_sym(sym1)
    sim1,psym1=syntactic_similarity(sym1,all_symp_pr)
    if sim1==1:
        psym1=related_sym(psym1)
    
    #get the 2nd syp ->> process it ->> check_pattern ->>> get the appropriate one (if check_pattern==1 == similar syntaxic symp found)

    print("Enter a second symptom you are experiencing Mr/Ms "+name+"  \n\t\t\t\t\t\t",end="=>")
    sym2=input("")
    sym2=preprocess_sym(sym2)
    sim2,psym2=syntactic_similarity(sym2,all_symp_pr)
    if sim2==1:
        psym2=related_sym(psym2)
        
    #if check_pattern==0 no similar syntaxic symp1 or symp2 ->> try semantic similarity
    
    if sim1==0 or sim2==0:
        sim1,psym1=semantic_similarity(sym1,all_symp_pr)
        sim2,psym2=semantic_similarity(sym2,all_symp_pr)
        
        #if semantic sim syp1 ==0 (no symp found) ->> suggest possible data symptoms based on all data and input sym synonymes
        if sim1==0:
            sugg=suggest_syn(sym1)
            print('Are you experiencing any ')
            for res in sugg:
                print(res)
                inp=input('')
                if inp=="yes":
                    psym1=res
                    sim1=1
                    break
                
        #if semantic sim syp2 ==0 (no symp found) ->> suggest possible data symptoms based on all data and input sym synonymes
        if sim2==0:
            sugg=suggest_syn(sym2)
            for res in sugg:
                inp=input('Do you feel '+ res+" ?(yes or no) ")
                if inp=="yes":
                    psym2=res
                    sim2=1
                    break
        #if no syntaxic semantic and suggested sym found return None and ask for clarification

        if sim1==0 and sim2==0:
            return None,None
        else:
            # if at least one sym found ->> duplicate it and proceed
            if sim1==0:
                psym1=psym2
            if sim2==0:
                psym2=psym1
    #create patient symp list
    all_sym=[col_dict[psym1],col_dict[psym2]]
    #predict possible diseases
    diseases=possible_diseases(all_sym)
    stop=False
    print("Are you experiencing any ")
    for dis in diseases:
        print(diseases)
        if stop==False:
            for sym in symVONdisease(df_tr,dis):
                if sym not in all_sym:
                    print(clean_symp(sym)+' ?')
                    while True:
                        inp=input("")
                        if(inp=="yes" or inp=="no"):
                            break
                        else:
                            print("provide proper answers i.e. (yes/no) : ",end="")
                    if inp=="yes":
                        all_sym.append(sym)
                        diseases=possible_diseases(all_sym)
                        if len(diseases)==1:
                            stop=True 
    return knn_clf.predict(OHV(all_sym,all_symp_col)),all_sym

    
    

In [85]:
def chat_sp():
    a=True
    while a:
        name=getInfo()
        result,sym=main_sp(name,all_symp_col)
        if result == None :
            ans3=input("can you specify more what you feel or tap q to stop the conversation")
            if ans3=="q":
                a=False
            else:
                continue

        else:
            print("you may have "+result[0])
            print(description_list[result[0]])
            an=input("how many days do you feel those symptoms ?")
            if calc_condition(sym,int(an))==1:
                print("you should take the consultation from doctor")
            else : 
                print('Take following precautions : ')
                for e in precautionDictionary[result[0]]:
                    print(e)
            print("do you need another medical consultation (yes or no)? ")
            ans=input()
            if ans!="yes":
                a=False
                print("!!!!! thanks for using our application !!!!!! ")


In [86]:
df_tr.iloc[-1]

itching                        0
skin_rash                      1
nodal_skin_eruptions           0
continuous_sneezing            0
shivering                      0
                          ...   
inflammatory_nails             0
blister                        1
red_sore_around_nose           1
yellow_crust_ooze              1
prognosis               Impetigo
Name: 4919, Length: 133, dtype: object

In [87]:
import joblib
knn_clf=joblib.load('knn.pkl')  

In [88]:
symVONdisease(df_tr,"Jaundice")

['itching',
 'vomiting',
 'fatigue',
 'weight_loss',
 'high_fever',
 'yellowish_skin',
 'dark_urine',
 'abdominal_pain']

In [89]:
knn_clf.predict(OHV(['fatigue', 'weight_loss', 'itching','high_fever'],all_symp_col))

array(['Jaundice'], dtype=object)

In [90]:
d=df_tr[df_tr.iloc[:,-1]=="Fungal infection"].sum(axis=0)

In [91]:
cl=df_tr.columns

In [92]:
pp=d!=0

In [93]:
cl[pp]

Index(['itching', 'skin_rash', 'nodal_skin_eruptions', 'dischromic _patches',
       'prognosis'],
      dtype='object')

In [94]:
d[pp].drop('prognosis')

itching                 108
skin_rash               108
nodal_skin_eruptions    108
dischromic _patches     108
dtype: object

In [95]:
chat_sp()

Your Name 
						=>Rudhreesh
hello  Rudhreesh
Enter the main symptom you are experiencing Mr/Ms Rudhreesh  
						=>cough
Enter a second symptom you are experiencing Mr/Ms Rudhreesh  
						=>headache
Are you experiencing any 
['Common Cold']
continuous sneezing ?
yes
chills ?
no
fatigue ?
yes
high fever ?
no
swelled lymph nodes ?
no
malaise ?
no
phlegm ?
no
throat irritation ?
no
redness of eyes ?
no
sinus pressure ?
no
runny nose ?
yes
congestion ?
no
chest pain ?
no
loss of smell ?
no
muscle pain ?
no
you may have Bronchial Asthma
Bronchial asthma is a medical condition which causes the airway path of the lungs to swell and narrow. Due to this swelling, the air path produces excess mucus making it hard to breathe, which results in coughing, short breath, and wheezing. The disease is chronic and interferes with daily working.
how many days do you feel those symptoms ?4
you should take the consultation from doctor
do you need another medical consultation (yes or no)? 
no
!!!!! thanks 